In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize 

# Matplotlib y módulos para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means 
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

In [ ]:
#ya hecho antes 
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
Una vez hecha la sopa de datos se traen solo los datos de la tabla ya que contienen los codigos postales de Toronto.

table = soup.find('table')
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)


In [ ]:
#tabla hecha antes
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0,"Neigh"],axis=1)
postalcode_list.head()
postalcode_list.shape

In [ ]:
Geospatial_Coordinates=pd.read_csv("Geospatial_Coordinates.csv")
coord_list=pd.merge(postalcode_list,Geospatial_Coordinates,on="Postal Code")
coord_list.head()

In [ ]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto